In [1]:
#Import Dependencies
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from splinter import Browser
import pandas as pd
import numpy as np
import regex as re
import datetime
import random
import time

In [2]:
def button_press(action):
    if action == "keystroke":
        return random.uniform(.101315816564,.25978136531)
    elif action == "enter_search":
        # This value needs to stay above 30 seconds to accommodate for the pause the website imposes when performing a search
        return random.uniform(31.1551845131843, 34.3189651876)
    else:
        return random.uniform(.3098741610684,.4598135130)

In [3]:
def get_dates(depart_day,trip_length,number_of_weeks):
    
    #Get today's date
    today = datetime.date.today()
    
    i = 0
    depart_ = []
    return_ = []
    for x in range(0,number_of_weeks):
        # Get the next possible departure date based on the desired day of the week to leave
        depart_.append((today + datetime.timedelta((depart_day-today.weekday()) % 7 )).strftime('%m/%d/%Y'))
        return_.append((today + datetime.timedelta((depart_day-today.weekday()) % 7 + trip_length)).strftime('%m/%d/%Y'))
        today = today + datetime.timedelta((depart_day-today.weekday()) % 7 + 7)
        i += 1
#         print(f"{depart_[i]} : {return_[i]}")
        
    return depart_,return_

In [4]:
def move_and_click(element):
    action.move_to_element(element).perform()
    time.sleep(button_press(0))
    
    action.click(element).perform()
    time.sleep(button_press(0))   

In [5]:
def set_time_of_day(element):
    
    # Select the element
    move_and_click(element)

#     # Move down to "Early Evening"
#     for x in range(0,4):
#         action.send_keys(Keys.DOWN)
#         time.sleep(button_press("keystroke"))
        
    # move up to "All Day"
    action.send_keys(Keys.UP)
    time.sleep(button_press("keystroke"))

    action.click(element).perform()
    time.sleep(button_press(0))

In [6]:
def enter_airport(airport,element):

    move_and_click(element)

    # Delete any 3-letter airport codes that may be currently in the field
    for x in range(0,3):
        action.send_keys(Keys.BACKSPACE).perform()
        time.sleep(button_press("keystroke"))

    # write in the new airport code
    for letter in airport:
        action.send_keys(letter).perform()
        time.sleep(button_press("keystroke"))

    time.sleep(button_press(0))  

In [7]:
def enter_dates(date,element):

    move_and_click(element)
    
    for letter in date:
        action.send_keys(letter).perform()
        time.sleep(button_press("keystroke"))
        
    time.sleep(button_press(0))  

In [8]:
def execute_search(from_airport,to_airport,depart_dates,return_dates):
    
    # Set the departure and arrival airports
    from_airport = "LAX"
    to_airport = "ORD"
    
    # Set the departure and return times
    depart_date = "06/23/2022"
    return_date = "06/27/2022"
    
    # Set Time of Day for Departure and Return
    set_time_of_day(driver.find_element(by=By.XPATH,value='//*[@id="segments0.travelTime"]'))
    set_time_of_day(driver.find_element(by=By.XPATH,value='//*[@id="segments1.travelTime"]'))

    # Set the travel dates
    enter_dates(depart_date,driver.find_element(by=By.XPATH,value='//*[@id="segments0.travelDate"]'))
    enter_dates(return_date,driver.find_element(by=By.XPATH,value='//*[@id="segments1.travelDate"]'))

    # Set the airports
    enter_airport(from_airport,driver.find_element(by=By.XPATH,value=('//*[@id="segments0.origin"]')))
    enter_airport(to_airport,driver.find_element(by=By.XPATH,value=('//*[@id="segments0.destination"]')))

    # Hitting the ENTER key while still in the airports field will execute a search
    action.send_keys(Keys.ENTER).perform()
    time.sleep(button_press("enter_search"))




In [9]:
def get_trip_length(x):
    x = re.findall("<li.*?>",str(x.prettify()))[0]
    x = re.findall("data-triptime=\".*?\"",x)[0]
    return float(re.findall("[0-9]+",x)[0])

In [10]:
def get_max_trip_length(list_elements):
    
    shortest_trip_length = 2**15 # minutes
    for x in list_elements:
        x = get_trip_length(x)
        if x < shortest_trip_length:
            shortest_trip_length = x
    
    # After finding the shortest trip, return a maximum trip length of 2x the shortest
    return shortest_trip_length * 2

In [11]:
def trip_duration_filter(list_elements):
    # Get the maximum trip length
    max_trip_length = get_max_trip_length(list_elements)
    
    i = 0
    duration_filtered_elements = []
    for x in list_elements:
        y = get_trip_length(x)
        if y < max_trip_length:
            duration_filtered_elements.append(x)
        i += 1
    
    return duration_filtered_elements

In [12]:
# create webdriver object
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# create action chain object
action = ActionChains(driver)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\morroe1\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\morroe1\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [13]:
# navigate to website
driver.get("https://www.aa.com/booking/find-flights")
time.sleep(button_press(0))

In [14]:
# Need a function for building lists of desired departure and return airports


In [15]:
from_airport = "LAX"
to_airport = "ORD"

In [16]:
depart_date = "06/24/2022"
return_date = "06/26/2022"

In [17]:
# Python calendar elements use "3" to represent Thursday
# Python calendar elements use "4" to represent Friday
depart_day = 3
trip_length = 3 # This value minus 1 gives the total number of non-travel days spent at the destination
number_of_weeks = 1
depart_dates,return_dates = get_dates(depart_day,trip_length,number_of_weeks)
print(depart_dates)

['06/23/2022']


In [18]:
for depart_date,return_date in depart_dates,return_dates"
    execute_search(from_airport,to_airport,depart_date,return_date)

In [19]:
# Get the html from the page
html = driver.page_source
html_soup = soup(html,'html.parser')

In [20]:
# Identifying the list elements that contain flight and price information
list_elements = html_soup.find_all("li",id=re.compile("slice0Flight[0-9]+"))

In [21]:
duration_filtered_elements = trip_duration_filter(html_soup.find_all("li",id=re.compile("slice0Flight[0-9]+")))

In [22]:
# Identifying the list elements that contain flight and price information
list_elements = duration_filtered_elements

In [23]:


# i = 0
# lowest_fare = 2**15
# lowest_fare_element = []
# for x in list_elements:
#     list_elements[i] = re.findall("<li.*?>",str(x.prettify()))[0]
#     new_lowest_fare = re.findall("data-lowestfare=\".*?\"",list_elements[i])[0]
#     new_lowest_fare = float(re.findall("[0-9]+",new_lowest_fare)[0])
#     if new_lowest_fare < lowest_fare:
#         lowest_fare = new_lowest_fare
#         lowest_fare_element.append(x)
# #     print(f"{i}: {list_elements[i]}")
# #     print(lowest_fare_element)
#     i += 1
    
i = 0
lowest_fare = 2**15
for x in list_elements:
    x = re.findall("<li.*?>",str(x.prettify()))[0]
    new_lowest_fare = re.findall("data-lowestfare=\".*?\"",x)[0]
    new_lowest_fare = float(re.findall("[0-9]+",new_lowest_fare)[0])
    if new_lowest_fare < lowest_fare:
        lowest_fare = new_lowest_fare
        lowest_fare_element = list_elements[i]
#     print(f"{i}: {list_elements[i]}")
    i += 1
print(lowest_fare_element)

lowest_fare_element = re.findall("<button.*?MainCabin.*?>",str(lowest_fare_element.prettify()))[0]
# print(lowest_fare_element)

lowest_fare_id = re.findall("id=\"slice\dFlight[0-9]+MainCabin\"",lowest_fare_element)[0]
# print(lowest_fare_id)
lowest_fare_id = f"//*[@{lowest_fare_id}]"
# print(lowest_fare_id)

lowest_fare_element = driver.find_element(
    by=By.XPATH,
    value=lowest_fare_id
)
print(lowest_fare_element)

<li class="flight-search-results js-moreflights" data-arrivaltime="06-23-2022 22:43:00" data-departuretime="06-23-2022 16:35:00" data-flightindex="5" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops="0" data-triptime="248" id="slice0Flight5">
<div class="row flight-matrix">
<div class="span6">
<div class="js-showmorefocus">
<div class="row-nobreak">
<div class="span4 span-phone6">
<span class="flight-airport-code">LAX</span>
</div>
<div class="span4 span-phone5">
<span class="flight-airport-code">ORD</span>
</div>
</div>
<div class="row-noBreak">
<div class="span3 span-phone4">
<span class="flight-time gamma aa-strong">
		4:35
		<span>PM</span>
</span>
</div>
<div class="span1 span-phone2">
<span aria-hidden="true" class="icon-arrow-right origin-destination-icon"></span>
</div>
<div class="span3 span-phone5">
<span class="flight-time gamma aa-strong">
		10:43
		<span>PM</span>
</span>
</div>
<div class="span2 hidden-phone">
<div class="flight-duration-stops margin-top -s

In [24]:
# sort the results by price, so that the lowest price is no longer hidden
sort_menu_element = driver.find_element(By.XPATH,value='//*[@id="searchResultsSortOptions"]')
move_and_click(sort_menu_element)

# Move down to "Price (low to high)"
for x in range(0,2):
    action.send_keys(Keys.DOWN)
    time.sleep(button_press("keystroke"))

action.click(sort_menu_element).perform()
time.sleep(button_press(0))
    
# Click the button for the lowest flight option
move_and_click(lowest_fare_element)

In [25]:
# Identifying the list elements that contain flight and price information for return flights
# Get the html from the page
html_return = driver.page_source
html_soup_return = soup(html_return,'html.parser')

return_elements = html_soup_return.find_all("li",id=re.compile("slice1Flight\d"))

i = 0
for x in return_elements:
#     list_elements[i] = re.search("<li\s*>\w",str(x.prettify()))
    return_elements[i] = re.findall("<li.*?>",str(x.prettify()))[0]
    print(f"{i}: {return_elements[i]}")
    i += 1

0: <li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 15:32:00" data-departuretime="06-27-2022 13:19:00" data-flightindex="1" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops="0" data-triptime="253" id="slice1Flight1">
1: <li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 12:18:00" data-departuretime="06-27-2022 10:05:00" data-flightindex="2" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops="0" data-triptime="253" id="slice1Flight2">
2: <li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 17:32:00" data-departuretime="06-27-2022 15:20:00" data-flightindex="3" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops="0" data-triptime="252" id="slice1Flight3">
3: <li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 10:35:00" data-departuretime="06-27-2022 08:20:00" data-flightindex="4" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops=

In [26]:
#This tool solely used in early dev for identifying html elemnts that contain information


# button_elements = html_soup.findAll('button')
# i = 0
# for x in button_elements:
#     print(f"{i}: {x}")
#     i += 1

In [27]:
# driver.close()

In [28]:
return_elements = html_soup_return.find_all("li",id=re.compile("slice1Flight\d"))
i = 0
for x in return_elements:
#     list_elements[i] = re.search("<li\s*>\w",str(x.prettify()))
#     return_elements[i] = re.findall("<li.*?>",str(x.prettify()))[0]
    return_elements[i] = re.findall("<li[^>]+flight-search-results[^>]+>",str(x))
    print(f"{i}: {return_elements[i]}")
    i += 1
    
return_elements = html_soup_return.find_all("li",id=re.compile("slice1Flight\d"))

0: ['<li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 15:32:00" data-departuretime="06-27-2022 13:19:00" data-flightindex="1" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops="0" data-triptime="253" id="slice1Flight1">']
1: ['<li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 12:18:00" data-departuretime="06-27-2022 10:05:00" data-flightindex="2" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops="0" data-triptime="253" id="slice1Flight2">']
2: ['<li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 17:32:00" data-departuretime="06-27-2022 15:20:00" data-flightindex="3" data-lowestfare="705.00" data-tripprice="705.00" data-tripstops="0" data-triptime="252" id="slice1Flight3">']
3: ['<li class="flight-search-results js-moreflights" data-arrivaltime="06-27-2022 10:35:00" data-departuretime="06-27-2022 08:20:00" data-flightindex="4" data-lowestfare="705.00" data-tripprice="705.00" d

In [29]:
'''
<li
data-arrivaltime="06-26-2022 08:25:00"
data-departuretime="06-26-2022 07:25:00"
data-flightindex="6"
data-lowestfare="666.00"
data-tripprice="666.00"
data-tripstops="0"
data-triptime="180"
'''
all_keys = [
    "data-arrivaltime",
    "data-departuretime",
    "data-flightindex",
    "data-lowestfare",
    "data-tripprice",
    "data-tripstops",
    "data-triptime",
]
# x.find('li')["aria-describedby"]
# return_elements[2].find_all('li', class_="flight-search-results")
dict_vals = []
for i in return_elements:
    a = {}
    for x in all_keys:
        a[x] = i[x]
    dict_vals.append(a)
#     print(return_elements[2][x])